## Commands
- To run Mongodb: 
    - terminal1: sudo mongod --dbpath /System/Volumes/Data/data/db
    - terminal2: mongo
    - terminal3: pyspark --conf "spark.mongodb.input.uri=mongodb://127.0.0.1/Quake.quakes?readPreference=primaryPreferred" --conf "spark.mongodb.output.uri=mongodb://127.0.0.1/Quake.quakes" --packages org.mongodb.spark:mongo-spark-connector_2.12:2.4.1
    
    
    Source: https://stackoverflow.com/questions/58034955/read-only-file-system-when-attempting-mkdir-data-db-on-mac

## Test if pyspark is working on Jupyter notebook

In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession


In [3]:
spark = SparkSession.builder.getOrCreate()

df = spark.sql("select 'spark' as hello")

df.show()

+-----+
|hello|
+-----+
|spark|
+-----+



## Data exploration

In [6]:
import findspark
findspark.init()

In [7]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *  #detect datasets
from pyspark.sql.functions import * #datatype conversion


# Configure spark session
spark = SparkSession \
        .builder \
        .master('local[2]') \
        .appName('quake_etl') \
        .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:2.4.1') \
        .getOrCreate()


In [8]:
# Load dataset
path = '/Users/clairehe/Documents/Data-pipeline-project/database.csv'
df_load = spark.read.csv(path, header = True)

# Preview data
df_load.take(1)

[Row(Date='01/02/1965', Time='13:44:18', Latitude='19.246', Longitude='145.616', Type='Earthquake', Depth='131.6', Depth Error=None, Depth Seismic Stations=None, Magnitude='6', Magnitude Type='MW', Magnitude Error=None, Magnitude Seismic Stations=None, Azimuthal Gap=None, Horizontal Distance=None, Horizontal Error=None, Root Mean Square=None, ID='ISCGEM860706', Source='ISCGEM', Location Source='ISCGEM', Magnitude Source='ISCGEM', Status='Automatic')]

In [9]:
# Drop fields we don't need 
lst_dropped_columns = ['Depth Error', 'Time', 'Depth Seismic Stations', 'Magnitude Error', 'Magnitude Seismic Stations', 'Azimuthal Gap', 'Horizontal Distance', 'Horizontal Error', 'Root Mean Square', 'Source', 'Location Source', 'Magnitude Source', 'Status']

df_load = df_load.drop(*lst_dropped_columns)
df_load.show(5)


+----------+--------+---------+----------+-----+---------+--------------+------------+
|      Date|Latitude|Longitude|      Type|Depth|Magnitude|Magnitude Type|          ID|
+----------+--------+---------+----------+-----+---------+--------------+------------+
|01/02/1965|  19.246|  145.616|Earthquake|131.6|        6|            MW|ISCGEM860706|
|01/04/1965|   1.863|  127.352|Earthquake|   80|      5.8|            MW|ISCGEM860737|
|01/05/1965| -20.579| -173.972|Earthquake|   20|      6.2|            MW|ISCGEM860762|
|01/08/1965| -59.076|  -23.557|Earthquake|   15|      5.8|            MW|ISCGEM860856|
|01/09/1965|  11.938|  126.427|Earthquake|   15|      5.8|            MW|ISCGEM860890|
+----------+--------+---------+----------+-----+---------+--------------+------------+
only showing top 5 rows



In [10]:
# Create a year field and add it to df
df_load = df_load.withColumn('Year', year(to_timestamp('Date', 'dd/MM/yyyy')))  #extract year value from date
df_load.show(5)


+----------+--------+---------+----------+-----+---------+--------------+------------+----+
|      Date|Latitude|Longitude|      Type|Depth|Magnitude|Magnitude Type|          ID|Year|
+----------+--------+---------+----------+-----+---------+--------------+------------+----+
|01/02/1965|  19.246|  145.616|Earthquake|131.6|        6|            MW|ISCGEM860706|1965|
|01/04/1965|   1.863|  127.352|Earthquake|   80|      5.8|            MW|ISCGEM860737|1965|
|01/05/1965| -20.579| -173.972|Earthquake|   20|      6.2|            MW|ISCGEM860762|1965|
|01/08/1965| -59.076|  -23.557|Earthquake|   15|      5.8|            MW|ISCGEM860856|1965|
|01/09/1965|  11.938|  126.427|Earthquake|   15|      5.8|            MW|ISCGEM860890|1965|
+----------+--------+---------+----------+-----+---------+--------------+------------+----+
only showing top 5 rows



In [11]:
# Build the quakes frequency dataframe using the year field and count for each year
df_quake_freq = df_load.groupBy('Year').count().withColumnRenamed('count', 'Counts')
df_quake_freq.show(5)

+----+------+
|Year|Counts|
+----+------+
|1990|   196|
|1975|   150|
|1977|   148|
|2003|   187|
|2007|   211|
+----+------+
only showing top 5 rows



In [12]:
# Preview df schema
df_load.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Depth: string (nullable = true)
 |-- Magnitude: string (nullable = true)
 |-- Magnitude Type: string (nullable = true)
 |-- ID: string (nullable = true)
 |-- Year: integer (nullable = true)



In [13]:
# Cast some feilds from string to numeric types
df_load = df_load.withColumn('Latitude', df_load['Latitude'].cast(DoubleType())) \
            .withColumn('Longitude', df_load['Longitude'].cast(DoubleType())) \
            .withColumn('Depth', df_load['Depth'].cast(DoubleType())) \
            .withColumn('Magnitude', df_load['Magnitude'].cast(DoubleType()))
df_load.show(5)

+----------+--------+---------+----------+-----+---------+--------------+------------+----+
|      Date|Latitude|Longitude|      Type|Depth|Magnitude|Magnitude Type|          ID|Year|
+----------+--------+---------+----------+-----+---------+--------------+------------+----+
|01/02/1965|  19.246|  145.616|Earthquake|131.6|      6.0|            MW|ISCGEM860706|1965|
|01/04/1965|   1.863|  127.352|Earthquake| 80.0|      5.8|            MW|ISCGEM860737|1965|
|01/05/1965| -20.579| -173.972|Earthquake| 20.0|      6.2|            MW|ISCGEM860762|1965|
|01/08/1965| -59.076|  -23.557|Earthquake| 15.0|      5.8|            MW|ISCGEM860856|1965|
|01/09/1965|  11.938|  126.427|Earthquake| 15.0|      5.8|            MW|ISCGEM860890|1965|
+----------+--------+---------+----------+-----+---------+--------------+------------+----+
only showing top 5 rows



In [14]:
# Preview df schema
df_load.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Type: string (nullable = true)
 |-- Depth: double (nullable = true)
 |-- Magnitude: double (nullable = true)
 |-- Magnitude Type: string (nullable = true)
 |-- ID: string (nullable = true)
 |-- Year: integer (nullable = true)



In [15]:
# Create avg magnitude and max magnitude fields and add to df_quake_freq
df_max = df_load.groupBy('Year').max('Magnitude').withColumnRenamed('max(Magnitude)', 'Max_Magnitude')
df_avg = df_load.groupBy('Year').avg('Magnitude').withColumnRenamed('avg(Magnitude)', 'Avg_Magnitude')

In [16]:
# Join df_max, df_avg, df_qiake_freq
df_quake_freq = df_quake_freq.join(df_avg, ['Year']).join(df_max, ['Year'])
df_quake_freq.show(5)

+----+------+-----------------+-------------+
|Year|Counts|    Avg_Magnitude|Max_Magnitude|
+----+------+-----------------+-------------+
|1990|   196|5.858163265306125|          7.6|
|1975|   150| 5.84866666666667|          7.8|
|1977|   148|5.757432432432437|          7.6|
|2003|   187|5.850802139037435|          7.6|
|2007|   211| 5.89099526066351|          8.4|
+----+------+-----------------+-------------+
only showing top 5 rows



In [17]:
# Remove nulls
df_load.dropna()
df_quake_freq.dropna()

DataFrame[Year: int, Counts: bigint, Avg_Magnitude: double, Max_Magnitude: double]

## Build dataframe into Mongodb

In [18]:
# Load database; if db dne, Mongodb will create a db
# In this case, mongodb create a db called "Quake" with table named "quakes", and stored every data inside df_load
df_load.write.format('mongo') \
    .mode('overwrite') \
    .option('spark.mongodb.output.uri', 'mongodb://127.0.0.1:27017/Quake.quakes').save()


In [19]:
# Write df_quake_freq to mongodb
df_quake_freq.write.format('mongo') \
    .mode('overwrite') \
    .option('spark.mongodb.output.uri', 'mongodb://127.0.0.1:27017/Quake.quake_freq').save()

## Machine Learning Data prepration

In [20]:
# Load test data file into df
path_test = '/Users/clairehe/Documents/Data-pipeline-project/query.csv'
df_test = spark.read.csv(path_test, header = True)
df_test.take(1)

[Row(time='2017-01-02T00:13:06.300Z', latitude='-36.0365', longitude='51.9288', depth='10', mag='5.7', magType='mwb', nst=None, gap='26', dmin='14.685', rms='1.37', net='us', id='us10007p5d', updated='2017-03-27T23:53:17.040Z', place='Southwest Indian Ridge', type='earthquake', horizontalError='10.3', depthError='1.7', magError='0.068', magNst='21', status='reviewed', locationSource='us', magSource='us')]

In [103]:
# Load training data from mongo into a dataframe
df_train = spark.read.format('mongo') \
        .option('spark.mongodb.input.uri', 'mongodb://127.0.0.1:27017/Quake.quakes').load()
df_train.show(5)

Py4JJavaError: An error occurred while calling o459.load.
: com.mongodb.MongoTimeoutException: Timed out after 30000 ms while waiting to connect. Client view of cluster state is {type=UNKNOWN, servers=[{address=127.0.0.1:27017, type=UNKNOWN, state=CONNECTING, exception={com.mongodb.MongoSocketReadTimeoutException: Timeout while receiving message}, caused by {java.net.SocketTimeoutException: Read timed out}}]
	at com.mongodb.internal.connection.BaseCluster.getDescription(BaseCluster.java:182)
	at com.mongodb.internal.connection.SingleServerCluster.getDescription(SingleServerCluster.java:41)
	at com.mongodb.client.internal.MongoClientDelegate.getServerAddressList(MongoClientDelegate.java:116)
	at com.mongodb.Mongo.getServerAddressList(Mongo.java:407)
	at com.mongodb.spark.connection.MongoClientCache.$anonfun$logClient$1(MongoClientCache.scala:161)
	at com.mongodb.spark.LoggingTrait.logInfo(LoggingTrait.scala:48)
	at com.mongodb.spark.LoggingTrait.logInfo$(LoggingTrait.scala:47)
	at com.m

In [23]:
# Select fields we will use and discard fields we dont need
df_test_clean = df_test['time', 'latitude', 'longitude', 'mag', 'depth'].withColumnRenamed('time', 'Date') \
                .withColumnRenamed('latitude', 'Latitude') \
                .withColumnRenamed('longitude', 'Longitude') \
                .withColumnRenamed('mag', 'Magnitude') \
                .withColumnRenamed('depth', 'Depth') 
df_test_clean.show(5)

+--------------------+--------+---------+---------+------+
|                Date|Latitude|Longitude|Magnitude| Depth|
+--------------------+--------+---------+---------+------+
|2017-01-02T00:13:...|-36.0365|  51.9288|      5.7|    10|
|2017-01-02T13:13:...|  -4.895| -76.3675|      5.9|   106|
|2017-01-02T13:14:...|-23.2513| 179.2383|      6.3|551.62|
|2017-01-03T09:09:...| 24.0151|  92.0177|      5.7|    32|
|2017-01-03T21:19:...|-43.3527| -74.5017|      5.5| 10.26|
+--------------------+--------+---------+---------+------+
only showing top 5 rows



In [24]:
df_test_clean = df_test_clean.withColumn('Latitude', df_test_clean['Latitude'].cast(DoubleType())) \
                    .withColumn('Longitude', df_test_clean['Longitude'].cast(DoubleType())) \
                    .withColumn('Depth', df_test_clean['Depth'].cast(DoubleType())) \
                    .withColumn('Magnitude', df_test_clean['Magnitude'].cast(DoubleType())) 

In [25]:
# Create training and testing dataframes
df_testing = df_test_clean['Latitude', 'Longitude', 'Magnitude', 'Depth']
df_training = df_train['Latitude', 'Longitude', 'Magnitude', 'Depth']

In [26]:
# Drop records with null values from our dataframe
df_testing = df_testing.dropna()
df_training = df_training.dropna()


## Machine learning

In [27]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator

In [28]:
# Select all features needed to parse into our model
assembler = VectorAssembler(inputCols = ['Latitude', 'Longitude', 'Depth'], outputCol = 'features')

# Create the model: use features to predict magnitude values
model_reg = RandomForestRegressor(featuresCol = 'features', labelCol = 'Magnitude')

# Chain assembler with the model in a pipeline
pipeline = Pipeline(stages = [assembler, model_reg])

# Train the model
model = pipeline.fit(df_training)

# Predict on test data
pred_results = model.transform(df_testing)

In [29]:
pred_results.show(5)

+--------+---------+---------+------+--------------------+-----------------+
|Latitude|Longitude|Magnitude| Depth|            features|       prediction|
+--------+---------+---------+------+--------------------+-----------------+
|-36.0365|  51.9288|      5.7|  10.0|[-36.0365,51.9288...|5.839371539930189|
|  -4.895| -76.3675|      5.9| 106.0|[-4.895,-76.3675,...|5.880300148595767|
|-23.2513| 179.2383|      6.3|551.62|[-23.2513,179.238...|5.896556405153653|
| 24.0151|  92.0177|      5.7|  32.0|[24.0151,92.0177,...|5.863755336563363|
|-43.3527| -74.5017|      5.5| 10.26|[-43.3527,-74.501...|5.934431053215677|
+--------+---------+---------+------+--------------------+-----------------+
only showing top 5 rows



## Evaluate ML model

In [76]:
# RMSE to avaluate
# rmse should be < 0.5 for the model to be useful

evaluator = RegressionEvaluator(labelCol = 'Magnitude', predictionCol = 'prediction', metricName = 'rmse')
rmse = evaluator.evaluate(pred_results)
print('RMSE = %g' % rmse)

RMSE = 0.403406


## Create prediction dataset

In [31]:
# Create the prediction dataset
df_pred_results = pred_results['Latitude', 'Longitude', 'prediction']

# Rename prediction fieldsk,mlj
df_pred_results = df_pred_results.withColumnRenamed('prediction', 'Pred_Magnitude')

# Add more columns to prediction dataset
df_pred_results = df_pred_results.withColumn('Year', lit(2017)) \
                    .withColumn('RMSE', lit(rmse))

df_pred_results.show(5)

+--------+---------+-----------------+----+-------------------+
|Latitude|Longitude|   Pred_Magnitude|Year|               RMSE|
+--------+---------+-----------------+----+-------------------+
|-36.0365|  51.9288|5.839371539930189|2017|0.40340590480815797|
|  -4.895| -76.3675|5.880300148595767|2017|0.40340590480815797|
|-23.2513| 179.2383|5.896556405153653|2017|0.40340590480815797|
| 24.0151|  92.0177|5.863755336563363|2017|0.40340590480815797|
|-43.3527| -74.5017|5.934431053215677|2017|0.40340590480815797|
+--------+---------+-----------------+----+-------------------+
only showing top 5 rows



In [32]:
# Load prediction dataset into mongodb
df_pred_results.write.format('mongo') \
    .mode('overwrite') \
    .option('spark.mongodb.output.uri', 'mongodb://127.0.0.1:27017/Quake.pred_results').save()


## Data visualization

In [33]:
import pandas as pd
from bokeh.io import output_notebook, output_file
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.models import Circle, ColumnDataSource, Grid, LinearAxis, Plot
from bokeh.models.tools import HoverTool
import math 
from math import pi
from bokeh.palettes import Category20c
from bokeh.tile_providers import CARTODBPOSITRON
from bokeh.themes import built_in_themes
from bokeh.io import curdoc
from pymongo import MongoClient
from bokeh.tile_providers import get_provider, Vendors 


In [35]:
# Create a custom read function to read data from mongodb into a df
def read_mongo(host = '127.0.0.1', port = 27017, username = None, password = None, db = 'Quake', collection = 'pred_results'):
    # Connect to mongodb
    mongo_uri = 'mongodb://{}:{}/{}.{}'.format(host, port, db, collection)  # {host}, {port}, {database}, {collection}
    conn = MongoClient(mongo_uri)
    db = conn[db]
    
    # Select all records from the collection 
    cursor = db[collection].find()
    
    # Create the dataframe
    df = pd.DataFrame(list(cursor))
    
    # Delete the _id field
    del df['_id']
    return df

In [36]:
# Load dataset from mongodb
df_quakes = read_mongo(collection = 'quakes')
df_quake_freq = read_mongo(collection = 'quake_freq')
df_quake_pred = read_mongo(collection = 'pred_results')

In [37]:
df_quakes_2016 = df_quakes[df_quakes['Year'] == 2016]
df_quakes_2016.head()

,Date,Latitude,Longitude,Type,Depth,Magnitude,Magnitude Type,ID,Year
22943,01/01/2016,-50.5575,139.4489,Earthquake,10.00,6.3,MWW,US10004ANT,2016.0
22944,01/01/2016,-28.6278,-177.2810,Earthquake,34.00,5.8,MWW,US10004AQY,2016.0
22945,01/02/2016,44.8069,129.9406,Earthquake,585.47,5.8,MWW,US10004ATB,2016.0
22946,01/03/2016,24.8036,93.6505,Earthquake,55.00,6.7,MWW,US10004B2N,2016.0
22947,01/05/2016,30.6132,132.7337,Earthquake,4.71,5.8,MWW,US10004BEN,2016.0


In [38]:
# Show plots embedded in jupyter notebook
output_notebook()

Loading BokehJS ...

In [51]:
# Create custom style function to style our plots
def style(p):
    # Title
    p.title.align='center'
    p.title.text_font_size='20pt'
    p.title.text_font='serif'
    
    # Axis titles
    p.xaxis.axis_label_text_font_size='14pt'
    p.xaxis.axis_label_text_font_style='bold'
    p.yaxis.axis_label_text_font_size='14pt'
    p.yaxis.axis_label_text_font_style='bold'
    
    # Tick labels
    p.xaxis.major_label_text_font_size='12pt'
    p.yaxis.major_label_text_font_size='12pt'
    
    # Plot the legend in the top left corner
    p.legend.location='top_left'
    
    return p

### Create Geomap plot


In [58]:
import json

from bokeh.io import output_file, show
from bokeh.models import GeoJSONDataSource
from bokeh.plotting import figure
from bokeh.sampledata.sample_geojson import geojson
from bokeh.tile_providers import get_provider, Vendors
output_file("geojson.html")

data = json.loads(geojson)
for i in range(len(data['features'])):
    data['features'][i]['properties']['Color'] = ['blue', 'red'][i%2]

geo_source = GeoJSONDataSource(geojson=json.dumps(data))

TOOLTIPS = [
    ('Organisation', '@OrganisationName')
]

p = figure(background_fill_color="lightgrey", tooltips=TOOLTIPS)
p.circle(x='x', y='y', size=15, color='Color', alpha=0.7, source=geo_source)

show(p)

In [81]:
# Create the Geo Map plot
def plotMap():
    lat = df_quakes_2016['Latitude'].values.tolist()
    lon = df_quakes_2016['Longitude'].values.tolist()
    
    pred_lat = df_quake_pred['Latitude'].values.tolist()
    pred_lon = df_quake_pred['Longitude'].values.tolist()
    
    lst_lat = []
    lst_lon = []
    lst_pred_lat = []
    lst_pred_lon = []
    
    i=0
    j=0
    
    # Convert Lat and Long values into merc_projection format
    for i in range(len(lon)):
        r_major = 6378137.000
        x = r_major * math.radians(lon[i])
        scale = x/lon[i]
        y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 +
            lat[i] * (math.pi/180.0)/2.0)) * scale
        
        lst_lon.append(x)
        lst_lat.append(y)
        i += 1
        
    # Convert predicted lat and long values into merc_projection format
    for j in range(len(pred_lon)):
        r_major = 6378137.000
        x = r_major * math.radians(pred_lon[j])
        scale = x/pred_lon[j]
        y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 +
            pred_lat[j] * (math.pi/180.0)/2.0)) * scale
        
        lst_pred_lon.append(x)
        lst_pred_lat.append(y)
        j += 1
    
    
    df_quakes_2016['coords_x'] = lst_lat
    df_quakes_2016['coords_y'] = lst_lon
    df_quake_pred['coords_x'] = lst_pred_lat
    df_quake_pred['coords_y'] = lst_pred_lon
    
    # Scale the circles
    df_quakes_2016['Mag_Size'] = df_quakes_2016['Magnitude'] * 4
    df_quake_pred['Mag_Size'] = df_quake_pred['Pred_Magnitude'] * 4
    
    # create datasources for our ColumnDataSource object
    lats = df_quakes_2016['coords_x'].tolist()
    longs = df_quakes_2016['coords_y'].tolist()
    mags = df_quakes_2016['Magnitude'].tolist()
    years = df_quakes_2016['Year'].tolist()
    mag_size = df_quakes_2016['Mag_Size'].tolist()
    
    pred_lats = df_quake_pred['coords_x'].tolist()
    pred_longs = df_quake_pred['coords_y'].tolist()
    pred_mags = df_quake_pred['Pred_Magnitude'].tolist()
    pred_year = df_quake_pred['Year'].tolist()
    pred_mag_size = df_quake_pred['Mag_Size'].tolist()
    
    # Create column datasource
    cds = ColumnDataSource(
        data=dict(
            lat=lats,
            lon=longs,
            mag=mags,
            year=years,
            mag_s=mag_size
        )
    )
    
    pred_cds = ColumnDataSource(
        data=dict(
            pred_lat=pred_lats,
            pred_long=pred_longs,
            pred_mag=pred_mags,
            year=pred_year,
            pred_mag_s=pred_mag_size
        )
    )
    
    # Tooltips
    TOOLTIPS = [
        ("Year", " @year"),
        ("Magnitude", " @mag"),
        ("Predicted Magnitude", " @pred_mag")
    ]
    
    # Create figure
    p = figure(title='Earthquake Map',
              plot_width=2300, plot_height=450,
              x_range=(-2000000, 6000000),
              y_range=(-1000000, 7000000),
              tooltips=TOOLTIPS)
    p.add_tile(get_provider(Vendors.CARTODBPOSITRON))
    
    p.circle(x='lon', y='lat', size='mag_s', color='#cc0000', alpha=0.7,
            source=cds, legend='Quakes 2016')
    
    # Add circles for our predicted earthquakes
    p.circle(x='pred_long', y='pred_lat', size='pred_mag_s', color='#ccff33', alpha=7.0,
            source=pred_cds, legend='Predicted Quakes 2017')
        
    # Style the map plot
    # Title
    p.title.align='center'
    p.title.text_font_size='20pt'
    p.title.text_font='serif'
    
    # Legend
    p.legend.location='bottom_right'
    p.legend.background_fill_color='black'
    p.legend.background_fill_alpha=0.8
    p.legend.click_policy='hide'
    p.legend.label_text_color='white'
    p.xaxis.visible=False
    p.yaxis.visible=False
    p.axis.axis_label=None
    p.axis.visible=False
    p.grid.grid_line_color=None
    mp = style(p)
    
    #how(p)
    
    return mp
    
plotMap()  

<ipython-input-81-735e444642aa>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_quakes_2016['coords_x'] = lst_lat
<ipython-input-81-735e444642aa>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_quakes_2016['coords_y'] = lst_lon
<ipython-input-81-735e444642aa>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

Figure(id='10761', ...)

In [82]:
# Create the Bar Chart
def plotBar():
    # Load the datasource 
    cds = ColumnDataSource(data=dict(
        yrs = df_quake_freq['Year'].values.tolist(),
        numQuakes = df_quake_freq['Counts'].values.tolist()
    ))
    
    # Tooltip
    TOOLTIPS = [
        ('Year', ' @yrs'),
        ('Number of earthquakes', ' @numQuakes')
    ]
    
    # Create a figure
    barChart = figure(title='Frequency of Earthquakes by Year',
                     plot_height=400,
                     plot_width=1150,
                     x_axis_label='Years',
                     y_axis_label='Number of Occurances',
                     x_minor_ticks=2,
                     y_range=(0, df_quake_freq['Counts'].max() + 100),
                     toolbar_location=None,
                     tooltips=TOOLTIPS)
    
    # Create a vertical bar 
    barChart.vbar(x='yrs', bottom=0, top='numQuakes',
                 color='#cc0000', width=0.75,
                 legend='Year', source=cds)
    
    # Style the bar chart
    barChart = style(barChart)
    
    #show(barChart)
    
    return barChart
    
    
plotBar()
 


Figure(id='10834', ...)

In [83]:
# Create a magnitude plot
def plotMagnitude():
    # Load the datasource
    cds = ColumnDataSource(data=dict(
        yrs = df_quake_freq['Year'].values.tolist(),
        avg_mag = df_quake_freq['Avg_Magnitude'].round(1).values.tolist(),
        max_mag = df_quake_freq['Max_Magnitude'].values.tolist()
    ))
    
    # Tooltip
    TOOLTIPS = [
        ('Year', ' @yrs'),
        ('Average Magnitude', ' @avg_mag'),
        ('Maximum Magnitude', ' @max_mag')
    ]
    
    # Create the figure
    mp = figure(title='Maximum and Average Magnitude by Year',
               plot_width=1150, plot_height=400,
               x_axis_label='Years',
               y_axis_label='Magnitude',
               x_minor_ticks=2,
               y_range=(5, df_quake_freq['Max_Magnitude'].max() + 1),
               toolbar_location=None,
               tooltips=TOOLTIPS)
    
    # Max Magnitude
    mp.line(x='yrs', y='max_mag', color='#cc0000', line_width=2, legend='Max Magnitude', source=cds)
    mp.circle(x='yrs', y='max_mag', color='#cc0000', size=8, fill_color='#cc0000', source=cds)
    
    # Average Magnitude 
    mp.line(x='yrs', y='avg_mag', color='yellow', line_width=2, legend='Avg Magnitude', source=cds)
    mp.circle(x='yrs', y='avg_mag', color='yellow', size=8, fill_color='yellow', source=cds)
    
    mp = style(mp)
    
    #show(mp)
    
    return mp

plotMagnitude()    

Figure(id='10888', ...)

In [84]:
# Display the visuals directly in the browser
output_file('dashboard.html')
# Change to a dark theme
curdoc().theme = 'dark_minimal'

In [102]:
# Build the grid plot
from bokeh.layouts import gridplot

# Make the grid
#grid = gridplot([[plotBar(), plotMagnitude()],[plotMap()]], plot_width=1000, plot_height=500)
grid = gridplot([[plotMap()], [plotBar(), plotMagnitude()]])

#grid = gridplot([[plotMap()], [plotBar()], [plotMagnitude()]])

# Shor the grid
show(grid)

<ipython-input-81-735e444642aa>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_quakes_2016['coords_x'] = lst_lat
<ipython-input-81-735e444642aa>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_quakes_2016['coords_y'] = lst_lon
<ipython-input-81-735e444642aa>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

In [134]:
from bokeh.io import output_file, show
from bokeh.layouts import row
from bokeh.plotting import figure

output_file("layout.html")

x = list(range(11))
y0 = x
y1 = [10 - i for i in x]
y2 = [i - 5 for i in x]

# create three plots
s1 = figure(plot_width=250, plot_height=250, background_fill_color="#fafafa")
s1.circle(x, y0, size=12, color="#53777a", alpha=0.8)

s2 = figure(plot_width=250, plot_height=250, background_fill_color="#fafafa")
s2.triangle(x, y1, size=12, color="#c02942", alpha=0.8)

s3 = figure(plot_width=250, plot_height=250, background_fill_color="#fafafa")
s3.square(x, y2, size=12, color="#d95b43", alpha=0.8)

# put the results in a row and show
#show(column(s1, s2, s3))


GlyphRenderer(id='26747', ...)

In [139]:
grid([s1, s2, s3], ncols=2)


TypeError: 'Column' object is not callable

In [104]:
l = 'hi=123'
l.split('=')

['hi', '123']

In [141]:
l

[2, 3]